In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [2]:
def get_url(position, location):
    temp = 'https://ie.indeed.com/jobs?q={}&l={}'
    url = temp.format(position, location)
    return url

def get_record(card):
    if card.h2.span.text == 'new':
        card.h2.span.decompose()
    job_title = card.h2.span.text
    job_url = 'https://ie.indeed.com' + card.get('href')
    try:
        company = card.find('span','companyName').text
    except AttributeError:
        company = ''
    location = card.find('div','companyLocation').text
    try:
        brief_jd = card.find('div','job-snippet').text.strip()
    except AttributeError:
        brief_jd = ''
    try:
        salary = card.find('div','salary-snippet').text
    except AttributeError:
        salary = ''
    post_date = card.find('span','date').text
    today = datetime.today().strftime('%d-%m-%Y')
    
    response2 = requests.get('https://ie.indeed.com' + card.get('href'))
    soup2 = BeautifulSoup(response2.text, 'html.parser')
    soup2.find('a', {'referrerpolicy' : 'origin'})
    try:
        app_url = soup2.find('a', {'referrerpolicy' : 'origin'})['href']
    except TypeError:
        app_url = ''
    except AttributeError:
        app_url = ''
    try:
        job_type = soup2.find('div', 'jobsearch-DesktopStickyContainer').find('span', 'jobsearch-JobMetadataHeader-item icl-u-xs-mt--xs').text
    except AttributeError:
        job_type = ''
    
    return (job_title, company, salary, location, job_type, brief_jd, job_url, app_url, post_date, today)

def scrape_indeed(position, location, industry, post_id):
    records = []
    url = get_url(position, location)
    
    while True:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        cards = soup.find_all("a", {'data-hide-spinner' : 'true'})
    
        for card in cards:
            record = get_record(card)
            records.append(record)
        try:
            url = 'https://ie.indeed.com' + soup.find('a', {'aria-label' : 'Next'}).get('href')
        except AttributeError:
            break
        filename = "%s.csv" % industry    
        if post_id == 0:    
            with open(filename, 'w', newline = '', encoding = 'utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(['JobTitle', 'Company', 'Salary', 'Location', 'JobType', 'JobDescription', 'JobURL', 'ApplicationURL', 'PostDate', 'ExtractDate'])
                writer.writerows(records)
        else:
            with open(filename, 'a', newline = '') as f:
                writer = csv.writer(f)
                writer.writerows(records)

In [ ]:
#industry = 'Food & Drink'
#position = ['chef', 'waiter', 'barman', 'bar manager', 'lounge staff', 'kitchen porter']

#industry = 'Childcare'
#position = ['childcare']

#industry = 'Fitness'
#position = ['fitness and leisure']

#industry = 'Grad'
#position = ['graduate']

#industry = 'Hair & Beauty'
#position = ['hair and beauty']

#industry = 'Hotel'
#position = ['hotel']

#industry = 'Manufacturing'
#position = ['manufacturing']

#industry = 'Market Research'
#position = ['market research']

#industry = 'Retail'
#position = ['retail']

#industry = 'Sales - €35k'
#position = ['sales €35k']

#industry = 'Secretarial - Admin - PA'
#position = ['secretarial', 'admin', 'pa']

#industry = 'Security'
#position = ['security']

#industry = 'Motor & Drivers'
#position = ['mechanic', 'mechanic apprenticeship', 'driver']

#industry = 'Support - Tech Support'
#position = ['support', 'tech support']

#industry = 'Trades - Operative - Manual'
#position = ['trades', 'operator', 'manual']

#industry = 'Voluntary - Charity Work'
#position = ['voluntary', 'charity']

industry = 'Warehouse - Logistics - Shipping'
position = ['warehouse', 'logistics', 'shipping']



#industries = ['Food & Drink', 'Fitness', 'Grad', 'Hair & Beauty', 'Hotel', 'Manufacturing', 'Market Research', 'Retail', 'Sales - €35k', 'Secretarial - Admin - PA', 'Security', 'Support - Tech Support', 'Trades - Operative - Manual', 'Voluntary - Charity Work', 'Warehouse - Logistics - Shipping']
#positions = [['chef', 'waiter', 'barman', 'bar manager', 'lounge staff', 'kitchen porter'], ['fitness and leisure'], ['graduate'], ['hair and beauty'], ['hotel'], ['manufacturing'], ['market research'], ['retail'], ['sales €35k'], ['secretarial', 'admin', 'pa'], ['security'], ['support', 'tech support'], ['trades', 'operator', 'manual'], ['voluntary', 'charity'], ['warehouse', 'logistics', 'shipping']]

#if __name__ == __main__:
#    while True:
#        for industry in industries:
#            ind=0
#            i=0
#            for post in positions[ind]:
#                scrape(post, 'Ireland', industry, i)
#                i+=1
#            ind+=1
#            wait_hours = 1
#            print(f'Waiting {wait_hours} hour...')
#            time.sleep(60*60*wait_hours)
            

i=0
for post in position:
    scrape_indeed(post, 'Ireland', industry, i)
    i+=1

In [120]:
industries = ['Food & Drink', 'Motor & Drivers', 'Childcare', 'Fitness', 'Grad', 'Hair & Beauty', 'Hotel', 'Manufacturing', 'Market Research', 'Retail', 'Sales - €35k', 'Secretarial - Admin - PA', 'Security', 'Support - Tech Support', 'Trades - Operative - Manual', 'Voluntary - Charity Work', 'Warehouse - Logistics - Shipping']
positions = [['chef', 'waiter', 'barman', 'bar manager', 'lounge staff', 'kitchen porter'], ['mechanic', 'mechanic apprenticeship', 'driver'], ['childcare'], ['fitness and leisure'], ['graduate'], ['hair and beauty'], ['hotel'], ['manufacturing'], ['market research'], ['retail'], ['sales €35k'], ['secretarial', 'admin', 'pa'], ['security'], ['support', 'tech support'], ['trades', 'operator', 'manual'], ['voluntary', 'charity'], ['warehouse', 'logistics', 'shipping']]

for i in range(0, len(industries)):
    industry = industries[i]
    posts = []
    print(industry)
    for j in range(0, len(positions[i])):
        posts.append(positions[i][j])
    for post_id in range(0, len(posts)):
        scrape_indeed(posts[post_id], 'Ireland', industry, post_id)

Food & Drink
Motor & Drivers
Childcare
Fitness
Grad
Hair & Beauty
Hotel
Manufacturing
Market Research
Retail
Sales - €35k
Secretarial - Admin - PA
Security
Support - Tech Support
Trades - Operative - Manual
Voluntary - Charity Work
Warehouse - Logistics - Shipping
